In [1]:
import plotly.graph_objects as go
from numpy import radians, array, zeros, linalg

import constants
from func.rotation_matrix import rotation_matrix
from func.transition_matrix import transition_matrix
from func.Graph import Graph
from func.coordinate_system_lower_and_upper import coordinate_system_lower_and_upper
from func.matrix_1_4 import matrix_1_4
from func.cross_normal_to_oz import cross_normal_to_oz

# Нахождение точки пересечения тяги и рычага тренажёра

## 1 Общая запись матричных преобразований

**С помощью ниже представленного матричного вычисления мы перемещаем И точку, И её локальную систему координат.**
$$

\Bigg(
  \begin{vmatrix}
  \ 1& \ 0& \ 0& \ Δx \\
  \ 0& \ 1& \ 0& \ Δy \\
  \ 0& \ 0& \ 1& \ Δz \\
  \ 0& \ 0& \ 0& \ 1 \\
\end{vmatrix}
\times
\begin{vmatrix}
  \ cos(ψ)& \ -sin(ψ)& \ 0& \ 0 \\
  \ sin(ψ)& \ cos(ψ)& \ 0& \ 0 \\
  \ 0& \ 0& \ 1& \ 0 \\
  \ 0& \ 0& \ 0& \ 1 \\
\end{vmatrix}\Bigg)
\times
\begin{vmatrix}
  \ x_0\\
  \ y_0\\
  \ z_0\\
  \ 0\\
\end{vmatrix}
= (A \times B) \times С = D,
$$

где A - матрица перемещения, B - матрица поворота, C - начальные координаты точки.

Важно отметить, что в работе нас будут интересовать именно **матрицы перемещений**, а не координаты точек.

## 2 Начальные координаты точки 
$$
\begin{vmatrix}
  \ x_0\\
  \ y_0\\
  \ z_0\\
  \ 0\\
\end{vmatrix} = C
$$

Найдём точку пересечения 1-ой группы тяга-рычаг. Пусть крепление рычага (нижняя платформа) изначально находится в центре глобальной СК, тогда $(x_0, y_0, z_0)$ = (0, 0, 0)

In [2]:
local_lower_CS = constants.LOCAL_LOWER_CS
local_upper_CS = constants.LOCAL_UPPER_CS
# Важно уточнить, что тут могут быть абсолютно любые координаты
# Ниже, Вы увидите это на примере отрисовки локальных систем координат

## 3 Создадим матрицы переноса точек крепления верхней и нижней платформы

$$
\begin{vmatrix}
  \ 1& \ 0& \ 0& \ Δx \\
  \ 0& \ 1& \ 0& \ Δy \\
  \ 0& \ 0& \ 1& \ Δz \\
  \ 0& \ 0& \ 0& \ 1 \\
\end{vmatrix}
\times
\begin{vmatrix}
  \ cos(ψ)& \ -sin(ψ)& \ 0& \ 0 \\
  \ sin(ψ)& \ cos(ψ)& \ 0& \ 0 \\
  \ 0& \ 0& \ 1& \ 0 \\
  \ 0& \ 0& \ 0& \ 1 \\
\end{vmatrix}= A \times B
$$

Угол ψ (в нашем случае) - это угол OZ на который мы хотим повернуть локальную СК ДО перемещения. Если Вы хотите сначала переместиться, а затем повернуть СК, то измените порядок умножения матриц. (Δx, Δy, Δz) - это линейные координаты, на которые необходимо сместить точку. 

Если у Вас несколько поворотов, просто перемножьте матрицы этих поворотов, т.е.:
$$
A \times (B_1 \times B_2 \times ...)
$$

(Δx, Δy, Δz) являются глобальными координатами точек. Запишем их в виде матриц.

In [3]:
# Координаты точек в глобальной СК 
offset_lower = constants.OFFSET_LOWER
offset_upper = constants.OFFSET_UPPER

В plotly OZ направленна вверх. Мы хотим, чтобы в нашей локальной СК OZ - это ось поворота рычага, а OY вертикальна (совпадает с OZ). Для этого сначала мы перемещаем точку, а затем поворачиваем её СК.

In [4]:
rotate_local_cs = constants.ROTATE_LOCAL_CS

Создадим матрицу поворота для поворота локальных СК

In [5]:
matrix_rotate_local_cs = rotation_matrix(radians(rotate_local_cs[3]), 
                                         radians(rotate_local_cs[4]), 
                                         radians(rotate_local_cs[5])) 

### 3.1 Нижняя плита

Создадим матрицы переноса для каждой точки нижней платформы. Они состоят из 2-х матриц: матрицы смещения точек в глобальных координатах и матрица поворота локальной СК

In [6]:
# Матрица переноса нижней плиты
matrix_move_lower_plate = zeros((6,4,4))

for i in range(6):
        # Запишем матрицы перемещений для каждого центра тяги в один массив
        matrix_move_lower_plate[i] = (transition_matrix(offset_lower[i][0], 
                                                        offset_lower[i][1], 
                                                        offset_lower[i][2]).dot(
                                    rotation_matrix(radians(offset_lower[i][3]), 
                                                    radians(offset_lower[i][4]), 
                                                    radians(offset_lower[i][5])))).dot(
                                    matrix_rotate_local_cs)

Чтобы найти координаты точек осталось только перемножить их матрицы на координаты точек в глобальной СК 

In [7]:
# Точки нижней плиты
lower_points = zeros((6,4,1))

# Координаты нижней плиты находим через перемножение матрицы перемещения на локальную СК
for i in range(6):
        lower_points[i] = matrix_move_lower_plate[i].dot(local_lower_CS)

### 3.2 Верхняя плита

Создадим матрицу перемещения для ЦЕНТРАЛЬНОЙ ВЕРХНЕЙ точки. Точки верхней платформы будут зависеть от её положения.

In [8]:
move_upper_CS = array((0, 0, 300, 0, 0, 0))

In [9]:
# Сначала поворачиваем, затем смещаем верхнюю центральную точку
matrix_move_upper_CS = transition_matrix(move_upper_CS[0], 
                                         move_upper_CS[1],
                                         move_upper_CS[2]).dot(
                            rotation_matrix(radians(move_upper_CS[3]), 
                                            radians(move_upper_CS[4]), 
                                            radians(move_upper_CS[5])))

Матрицы перемещения для верхний точек состоят из 3-х матриц: матрица смещения точек в глобальных координатах, матрицы поворота локальных СК и матрицы перемещения центральной точки.

In [10]:
# Матрица переноса верхней плиты
matrix_move_upper_plate = zeros((9,4,4))
# Точки верхней плиты
upper_points = zeros((9,4,1))

for i in range(6):
    # Матрица переноса для точек верхней плиты 
    matrix_move_upper_plate[i] = matrix_move_upper_CS.dot(
            (transition_matrix(offset_upper[i][0], 
                               offset_upper[i][1], 
                               offset_upper[i][2]).dot(
                                    rotation_matrix(radians(offset_upper[i][3]), 
                                                    radians(offset_upper[i][4]), 
                                                    radians(offset_upper[i][5])))).dot(
                                    matrix_rotate_local_cs))  
    # Перемещение и поворот точек верхней плиты
    upper_points[i] = matrix_move_upper_plate[i].dot(local_upper_CS) 

Матрицы и координаты получены. Отрисуем результат

In [11]:
fig = go.Figure()

graph = Graph(fig, lower_points, upper_points, move_upper_CS)
graph.upper_plate()
graph.lower_plate()
graph.title()
graph.decor()

fig.show()

### 3.3 Системы координат

Теперь разберёмся с локальными системами координат для всех точек. Выше, мы получили матрицы перемещения для каждой точки. Пусть глобальная система координат выглядит следующим образом:

In [12]:
global_coordinate_system = constants.GLOBAL_COORDINATE_SYSTEM

Нахождение итоговых локальных СК определяется очень просто: перемножаем полученные выше матрицы переноса на **global_coordinate_system**.

P.s. откройте файл func.coordinate_system_lower_and_upper.py 

In [13]:
global_coordinate_system_upper_plate, local_coordinate_system_lower_point, local_coordinate_system_upper_point = coordinate_system_lower_and_upper(matrix_move_lower_plate, matrix_move_upper_plate, matrix_move_upper_CS, global_coordinate_system)

В результате мы получили СК центра плиты и всех точек плиты (для нижней плиты используем глобальную СК):

In [14]:
graph.coordinate_system(global_coordinate_system_upper_plate, 7, 1)
graph.coordinate_system_lower_plate()

for i in range(6):
    graph.coordinate_system(local_coordinate_system_lower_point[i], i)
    graph.coordinate_system(local_coordinate_system_upper_point[i], i)

fig.show()

Главная сложность в отрисовке локальных СК - это то, что в одной СК 4 точки, т.е. одна СК - это 4 матрицы => необходимо перемножать матрицы переноса точек платформы на ВСЕ 4 координаты СК (и тут бы отлично сработало ООП).

## 4 Точка пересечения тяги и рычага

Для нахождения точки пересечения необходимо определить координаты верхней точки относительно нижней (сейчас у нас имеются только глобальные координаты). Для этого мы перемножаем обратную матрицу перемещения нижней плиты на глобальные координаты верхней плиты.

In [15]:
# Координаты верхний точек относительно нижних
position_upper_points_from_lower = zeros((6, 4, 1))

for i in range(6):
    position_upper_points_from_lower[i] = linalg.inv(
                matrix_move_lower_plate[i]).dot(upper_points[i])

Теперь остаётся решить задачу пересечения окружности и сферы из 2. Sphere&circle.ipynb.

In [16]:
R = constants.R_CIRCLE
r = constants.R_SPHERE

# Координаты точек пересечения в локальных координатах
local_cross_coordinates = zeros((6, 4, 1))
# Углы рычагов
arm_angle = zeros(6)

for i in range(6):
    cross_matrix = cross_normal_to_oz(
                position_upper_points_from_lower[i], R, r)
    
    local_cross_coordinates[i] = matrix_1_4(
                cross_matrix[0], cross_matrix[1], cross_matrix[2])
    arm_angle[i] = cross_matrix[3]

cross_matrix - матрица 4х1, где первые 3 числа - это локальные координаты точки пересечения, а последнее число - угол.

Для перевода локальных координат в глобальные необходимо перемножить матрицу перемещения нижней платформы на локальные координаты.

In [17]:
# Координаты точек пересечения в глобальных координатах
global_cross_coordinates = zeros((6, 4, 1))

for i in range(6):
    global_cross_coordinates[i] = matrix_move_lower_plate[i].dot(
                local_cross_coordinates[i])

Отрисуем получаемый результат

In [18]:
for i in range(6):
    graph.legs(i, f"group{i}", arm_angle, global_cross_coordinates)
    
fig.show()